# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load CSV from WeatherPy
weather_info = pd.read_csv("../WeatherPy/output_data.csv")
weather_info


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,49.66,76,40,17.27,AR,1645158245
1,Kaitangata,-46.2817,169.8464,68.94,62,43,4.41,NZ,1645158246
2,Saint-Philippe,-21.3585,55.7679,83.75,79,87,13.96,RE,1645158246
3,Ternate,0.8000,127.4000,82.76,72,88,4.41,ID,1645158247
4,Atuona,-9.8000,-139.0333,79.00,74,7,18.90,PF,1645158247
...,...,...,...,...,...,...,...,...,...
862,Burladingen,48.2833,9.1167,41.83,74,97,5.39,DE,1645158577
863,Bulgan,48.8125,103.5347,3.88,79,28,3.06,MN,1645158578
864,Baijiantan,45.6333,85.1833,9.79,85,92,7.90,CN,1645158578
865,Ust-Maya,60.4167,134.5333,-10.41,87,93,0.81,RU,1645158579


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

location = weather_info[["Latitude", "Longitude"]]

humidity = weather_info["Humidity"]

In [6]:
#Heatmap time!

figure = gmaps.figure(center=(46.0,-5.0), zoom_level = 3)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating=False, max_intensity = 100, point_radius = 3)
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Narrow down cities to specific weather conditions
ideal_df = weather_info.loc[(weather_info["Wind Speed"]<= 10)& (weather_info["Cloudiness"] ==0) & (weather_info["Max Temperature"]>=70) & (weather_info ["Max Temperature"]<= 80)].dropna()

ideal_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
13,Pāthardih,23.6667,86.4333,73.00,27,0,2.89,IN,1645158251
321,Pāthardih,23.6667,86.4333,73.00,27,0,2.89,IN,1645158251
622,Conde,-7.2597,-34.9075,76.46,94,0,2.30,BR,1645158481
644,Nandurbar,21.3667,74.2500,77.14,37,0,8.68,IN,1645158490
721,Wajir,1.7471,40.0573,75.16,78,0,0.00,KE,1645158521
754,Neuquén,-38.9516,-68.0591,78.69,27,0,9.22,AR,1645158021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Hotel Map
hotel_df = ideal_df.loc[:, ["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
13,Pāthardih,IN,23.6667,86.4333,
321,Pāthardih,IN,23.6667,86.4333,
622,Conde,BR,-7.2597,-34.9075,
644,Nandurbar,IN,21.3667,74.2500,
721,Wajir,KE,1.7471,40.0573,
754,Neuquén,AR,-38.9516,-68.0591,


In [10]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Specify Parameters
params = {"type" : "hotel",
         "keyword" : "hotel",
        "radius" : 5000,
          "key" : g_key
         }

In [12]:
for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city = row["City"]
    
    params["location"] = f"{latitude},{longitude}"
    
    response = requests.get (url, params = params).json()
    
    results = response["results"]
    
    try:
        print(f"Closest hotel in{city} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No Result")

        

print("End")

No Result
No Result
No Result
Closest hotel inNandurbar is Hotel Heera Executive
Closest hotel inWajir is Wajir Palace Hotel
Closest hotel inNeuquén is Casino Magic Hotel & Casino
End


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))